In [4]:
# Import necessary libraries:
# We import numpy for numerical operations and scipy.sparse for sparse matrix handling.
import numpy as np
from scipy.sparse import csr_matrix, dok_matrix

# This function creates a transition matrix from the given link structure. It initializes a sparse matrix transition_matrix with the shape (num_nodes, num_nodes) 
# and iterates through the provided links. For each link, it calculates the transition probability (number of outbound links) and populates the transition matrix. 
# Finally, the function converts the sparse matrix to a Compressed Sparse Row (CSR) format for efficient matrix operations and returns it.
def create_transition_matrix(links, num_nodes):
    # Initialize a sparse matrix to store the transition probabilities
    transition_matrix = dok_matrix((num_nodes, num_nodes), dtype=np.float32)
    
    # Populate the transition matrix with the probabilities based on outbound links
    for from_node, to_nodes in links.items():
        num_outbound_links = len(to_nodes)
        for to_node in to_nodes:
            transition_matrix[to_node, from_node] = 1 / num_outbound_links

    # Convert to CSR format for efficient matrix operations
    return transition_matrix.tocsr()

In [5]:
# This function computes the PageRank vector for the given transition matrix. It takes the following parameters:
# transition_matrix: The transition matrix created earlier.
# damping_factor: A damping factor, usually set to 0.85, which accounts for random jumps between nodes.
# max_iterations: The maximum number of iterations allowed for the iterative eigenvector computation.
# tol: The tolerance level to determine convergence.
# The function initializes a rank vector with equal probabilities for each node and identifies dangling nodes (nodes with no outbound links). It then iteratively 
# updates the rank vector using the power method until the difference between the current rank vector and the previous rank vector is less than the tolerance or the
# maximum number of iterations is reached. The rank vector is normalized at each step, and the damping factor is applied to account for random jumps between nodes.

def page_rank(transition_matrix, num_nodes, damping_factor=0.85, max_iterations=100, tol=1e-6):
    # Initialize the rank vector with equal probabilities for each node
    rank_vector = np.ones(num_nodes) / num_nodes
    
    # Identify dangling nodes (nodes with no outbound links)
    dangling_nodes = np.ravel(np.array(transition_matrix.sum(axis=0) == 0))

    # Iterate until convergence or the maximum number of iterations is reached
    for _ in range(max_iterations):
        prev_rank_vector = rank_vector.copy()

        # Update the rank vector considering dangling nodes
        rank_vector = damping_factor * (transition_matrix @ rank_vector + np.sum(rank_vector[dangling_nodes]) / num_nodes)

        # Apply the damping factor and normalize the rank vector
        rank_vector = (1 - damping_factor) / num_nodes + rank_vector

        # Check for convergence
        if np.linalg.norm(rank_vector - prev_rank_vector) < tol:
            break

    return rank_vector

In [6]:
# In the main section, the network of nodes and their outbound links is defined using a dictionary. The number of nodes is calculated, and the transition matrix is
# created using the create_transition_matrix function. The PageRank vector is then computed using the page_rank function, and the final PageRank values are printed.
if __name__ == "__main__":
    # Define the network with node IDs as keys and lists of connected node IDs as values
    links = {
        0: [1, 2],
        1: [2],
        2: [0, 1],
        3: [2],
    }

    # Get the number of nodes in the network
    num_nodes = len(links)
    
    # Create the transition matrix from the network
    transition_matrix = create_transition_matrix(links, num_nodes)
    
    # Compute the PageRank vector using the iterative method
    rank_vector = page_rank(transition_matrix, num_nodes)

    print("PageRank values:", rank_vector)

PageRank values: [0.21991393 0.3133772  0.42920887 0.0375    ]


In summary, this code creates a transition matrix from a given network of nodes, computes the PageRank vector using the iterative eigenvector computation, and handles dangling nodes and random jumps using the damping factor.